# Supervisor Agent Testing 

Can we build an agent that can:
- Determine what transformation pipeline a given input should go to
- Schedule + run that job for us?

While this usecase isn't particularly compelling given we have 2 sources of reliably structured data, the broader applications are interesting. Think about the case of unstructured data. It would be unreasonable to write special conditions for every edge case you come across, however, an LLM could assist in making a reasonable guess as to which transformation pipline might be the best fit!

In [1]:
import sys 
import subprocess

# get root of current repo and add to our path
root_dir = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], stderr=subprocess.DEVNULL).decode("utf-8").strip()

sys.path.append(root_dir)

## Building the Supervisor Agent

Now that we have an S3 Tool to grab raw data + extract metadata and an Airflow Tool that can help us trigger DAGs via TaskFlow API, we're ready to build our agent!

In [8]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from airflow.dags.utils.airflow_client import AirflowClient

# define the set of available tools for our agent
tools = [
    Tool(
        name="TriggerAirflowDAG", 
        func=lambda dag_id: AirflowClient.trigger_dag(dag_id=dag_id),
        description="Trigger an Airflow DAG. The DAG is specified by its ID."
    )
]

In [9]:
# set the language model for our agent
llm = ChatOpenAI(model="gpt-4", temperature=0)

# initialize the agent
agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [15]:
from airflow.dags.utils.aws import S3 
import os 

def run_supervisor_agent(date):

    # get a list of s3 keys for a specific dates scrape 
    s3_keys = S3.get_keys_by_date(date=date, role_arn=os.getenv("DIGI_INNO_ROLE_ARN"))

    # get file meta 
    metadata = S3.get_file_metadata(key=s3_keys[0], chars_to_sample=500, role_arn=os.getenv("DIGI_INNO_ROLE_ARN"))
    
    prompt = f"""
        You are a supervisor agent. Your task is to examine the file content and decide which Airflow DAG should be triggered.

        Here is the metadata:
        {metadata}

        Choose the correct DAG ID from:
        - transform_pbs_data
        - transform_npr_data

        Then call TriggerAirflowDAG with the correct DAG ID.
    """

    print(agent.run(prompt))

In [16]:
run_supervisor_agent("05222025")



> Entering new AgentExecutor chain...
[2025-06-11T14:50:20.621-0400] {_client.py:1026} INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The file content seems to be an XML file from NPR (National Public Radio) as indicated by the URL and the XML namespaces. Therefore, the appropriate DAG to trigger would be the one related to NPR data.

Action: TriggerAirflowDAG
Action Input: transform_npr_data
Observation: {'status_code': 200, 'response': {'dag_run_id': 'manual__2025-06-11T18:50:20.627438+00:00', 'dag_id': 'transform_npr_data', 'logical_date': '2025-06-11T18:50:20.627438Z', 'queued_at': '2025-06-11T18:50:20.879650Z', 'start_date': None, 'end_date': None, 'data_interval_start': '2025-06-11T18:50:20.627438Z', 'data_interval_end': '2025-06-11T18:50:20.627438Z', 'run_after': '2025-06-11T18:50:20.627438Z', 'last_scheduling_decision': None, 'run_type': 'manual', 'state': 'queued', 'triggered_by': 'rest_api', 'conf': {}, 'note': None, 'dag_versions': [